In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/파이썬머신러닝완벽가이드

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

cust_df = pd.read_csv('train.csv',encoding = 'latin-1')
print('dataset shape : ',cust_df.shape)
cust_df.head(3)

In [ ]:
cust_df.info()

In [ ]:
print(cust_df['TARGET'].value_counts())
unsatisfied_cnt = cust_df[cust_df['TARGET'] == 1].TARGET.count()
total_cnt = cust_df.TARGET.count()
print(f'unsatisfied 비율은 {unsatisfied_cnt / total_cnt :.2f}')
# 0이 만족 1이 불만족

In [ ]:
cust_df.describe()

In [ ]:
print(cust_df.var3.value_counts()[:10])

In [ ]:
cust_df['var3'].replace(-999999,2,inplace = True)
cust_df.drop('ID',axis = 1,inplace = True)

x_features = cust_df.iloc[:,:-1]
y_labels = cust_df.iloc[:,-1]
print(f'피처 데이터 shape : {x_features.shape}')

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_features,y_labels,test_size = 0.2,random_state = 0)

train_cnt = y_train.count()
test_cnt = y_test.count()

print(f'학습 세트 Shape : {x_train.shape}, 테스트 세트 shape {x_test.shape}')

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 갓 분포 비율')
print(y_test.value_counts()/ test_cnt)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimators = 500,random_state = 156)

xgb_clf.fit(x_train,y_train,early_stopping_rounds = 100,eval_metric = 'auc',eval_set = [(x_train,y_train),(x_test,y_test)])

xgb_roc_score = roc_auc_score(y_test,xgb_clf.predict_proba(x_test)[:,1],average = 'macro')
print(f'ROC AUC {xgb_roc_score:.4f}')

In [ ]:
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(n_estimators = 500)

evals = [(x_test,y_test)]
lgbm_clf.fit(x_train,y_train,early_stopping_rounds = 100,eval_metric = 'auc',eval_set = evals,verbose = True)

lgbm_roc_score = roc_auc_score(y_test,lgbm_clf.predict_proba(x_test)[:,1],average='macro')
print(f'ROC AUC : {lgbm_roc_score:.4f}')

In [ ]:
from sklearn.model_selection import GridSearchCV

lgbm_clf = LGBMClassifier(n_estimators = 200)

params = {'num_leaves' : [32,64],
          'max_depth' : [128,160],
          'min_child_samples' : [60,100],
          'subsamples' : [0.8,1]}

gridcv = GridSearchCV(lgbm_clf,param_grid = params ,cv = 3)
gridcv.fit(x_train,y_train,early_stopping_rounds = 30,eval_metric = 'auc',
           eval_set = [(x_train,y_train),(x_test,y_test)])




In [ ]:
print('GridSearchCV 최적 파라미터 : ',gridcv.best_params_)
lgbm_roc_score = roc_auc_score(y_test,gridcv.predict_proba(x_test)[:,1],average = 'macro')
print(f'ROC AUC : {lgbm_roc_score:.4f}')

In [ ]:
lgbm_clf = LGBMClassifier(n_estimators = 1000,num_leaves = 32,subsample=0.8,min_child_samples=100,max_depth = 128)
evals = [(x_test,y_test)]

lgbm_clf.fit(x_train,y_train,early_stopping_rounds = 100, eval_metric = 'auc',eval_set = evals, verbose = True)

lgbm_roc_score = roc_auc_score(y_test,lgbm_clf.predict_proba(x_test)[:,1],average = 'macro')
print('ROC AUC : {0:.4f}'.format(lgbm_roc_score))